In [1]:
# step01 : Variable 클래스 구현
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data

data = np.array(1.0)
x = Variable(data)
print(x.data)

x.data = np.array(2.0)
print(x.data)

# 보충
print("--- 보충 ---")
x = np.array(1)
print(x.ndim)

x = np.array([1, 2, 3])
print(x.ndim)

x = np.array([[1, 2, 3], [4, 5, 6]])
print(x.ndim)

1.0
2.0
--- 보충 ---
0
1
2


In [2]:
# step02 : Function 클래스 구현
import numpy as np

class Function:
  def __call__(self, input):
    x = input.data  # 1. 데이터를 꺼냄
    y = self.forward(x)
    output = Variable(y)  # 2. Variable 형태로 돌림
    return output
  
  def forward(self, x):
    raise NotImplementedError()

class Square(Function): # Function 상속받음
  def forward(self, x):
    return x ** 2

x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [3]:
# step03 : 함수 연결
class Exp(Function):
  def forward(self, x):
    return np.exp(x)

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


In [4]:
# step04 : 수치 미분
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

f = Square()  # x**2
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

# 합성 함수의 미분
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

# 수치 미분 문제점 : 계산량 많음 -> 역전파

4.000000000004
3.2974426293330694


In [5]:
# step05 : 역전파 이론

In [8]:
# step06 : 수동 역전파
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None  # 추가


class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input  # 입력 변수 input을 기억
    return output
  
  def forward(self, x):
    raise NotImplementedError()
  
  def backward(self, x):  # 추가
    raise NotImplementedError()


class Square(Function): 
  def forward(self, x):
    return x ** 2
  
  def backward(self, gy): # 추가
    x = self.input.data
    gx = 2 * x * gy
    return gx


class Exp(Function):
  def forward(self, x):
    return np.exp(x)
  
  def backward(self, gy): # 추가
    x = self.input.data
    gx = np.exp(x) * gy
    return gx


A = Square()
B = Exp()
C = Square()

# forward
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0) # 역전파는 dy/dy = 1에서 시작
b.grad = C.backward(y.grad) # dy/db
a.grad = B.backward(b.grad) # dy/da = dy/db * db/da
x.grad = A.backward(a.grad) # dy/dx = dy/da * da/dx
print(x.grad)

3.297442541400256
